
---

## **1. Understanding the Scenario**

Imagine you work in a retail company, and you have a sales table:

### Sample Input Table: `sales`

| region | month | sales\_amount |
| ------ | ----- | ------------- |
| East   | Jan   | 1000          |
| East   | Feb   | 1200          |
| East   | Mar   | 900           |
| West   | Jan   | 700           |
| West   | Feb   | 1100          |
| West   | Mar   | 1300          |

**Goal:**

* **Pivot:** Convert months into columns with their sales totals per region.
* **Unpivot:** Convert month columns back into row form.

---

## **2. PIVOT in PostgreSQL (No Native Keyword)**

### **Method 1 — Using `FILTER` with `GROUP BY`**

```sql
SELECT
    region,
    SUM(sales_amount) FILTER (WHERE month = 'Jan') AS Jan_sales,
    SUM(sales_amount) FILTER (WHERE month = 'Feb') AS Feb_sales,
    SUM(sales_amount) FILTER (WHERE month = 'Mar') AS Mar_sales
FROM sales
GROUP BY region
ORDER BY region;
```

**Output:**

| region | Jan\_sales | Feb\_sales | Mar\_sales |
| ------ | ---------- | ---------- | ---------- |
| East   | 1000       | 1200       | 900        |
| West   | 700        | 1100       | 1300       |

**Step-by-step:**

1. `SUM(sales_amount)` → adds up sales.
2. `FILTER (WHERE month = 'Jan')` → only sums January sales.
3. Repeat for Feb, Mar.
4. `GROUP BY region` → one row per region.

---

### **Method 2 — Using `CASE WHEN` with `GROUP BY`**

```sql
SELECT
    region,
    SUM(CASE WHEN month = 'Jan' THEN sales_amount ELSE 0 END) AS Jan_sales,
    SUM(CASE WHEN month = 'Feb' THEN sales_amount ELSE 0 END) AS Feb_sales,
    SUM(CASE WHEN month = 'Mar' THEN sales_amount ELSE 0 END) AS Mar_sales
FROM sales
GROUP BY region
ORDER BY region;
```

Output is same as above.
**Why use this?** Works in all SQL databases, not just PostgreSQL.

---

### **Method 3 — Using `crosstab()` (Dynamic)**

```sql
-- Step 1: Enable tablefunc extension
CREATE EXTENSION IF NOT EXISTS tablefunc;

-- Step 2: Use crosstab
SELECT *
FROM crosstab(
    'SELECT region, month, sales_amount
     FROM sales
     ORDER BY region, month',
    $$ VALUES ('Jan'), ('Feb'), ('Mar') $$
) AS ct(region TEXT, Jan_sales INT, Feb_sales INT, Mar_sales INT);
```

**Output:**

| region | Jan\_sales | Feb\_sales | Mar\_sales |
| ------ | ---------- | ---------- | ---------- |
| East   | 1000       | 1200       | 900        |
| West   | 700        | 1100       | 1300       |

**When to use:** If your months are dynamic and you want them as columns without manually writing each one.

---

## **3. UNPIVOT in PostgreSQL**

Let’s say we already have the pivoted table:

### Input Table: `sales_pivot`

| region | Jan\_sales | Feb\_sales | Mar\_sales |
| ------ | ---------- | ---------- | ---------- |
| East   | 1000       | 1200       | 900        |
| West   | 700        | 1100       | 1300       |

---

### **Method 1 — Using `UNION ALL`**

```sql
SELECT region, 'Jan' AS month, Jan_sales AS sales_amount
FROM sales_pivot
UNION ALL
SELECT region, 'Feb', Feb_sales
FROM sales_pivot
UNION ALL
SELECT region, 'Mar', Mar_sales
FROM sales_pivot
ORDER BY region, month;
```

**Output:**

| region | month | sales\_amount |
| ------ | ----- | ------------- |
| East   | Jan   | 1000          |
| East   | Feb   | 1200          |
| East   | Mar   | 900           |
| West   | Jan   | 700           |
| West   | Feb   | 1100          |
| West   | Mar   | 1300          |

---

### **Method 2 — Using `jsonb_each_text()` for Dynamic Unpivot**

```sql
SELECT 
    region,
    REPLACE(key, '_sales', '') AS month,
    value::INT AS sales_amount
FROM sales_pivot,
LATERAL jsonb_each_text(to_jsonb(sales_pivot) - 'region');
```

**Output:**

| region | month | sales\_amount |
| ------ | ----- | ------------- |
| East   | Jan   | 1000          |
| East   | Feb   | 1200          |
| East   | Mar   | 900           |
| West   | Jan   | 700           |
| West   | Feb   | 1100          |
| West   | Mar   | 1300          |

**Why use:** Works without hardcoding months.

---

## **4. When to Use Pivot / Unpivot**

* **Pivot** → For dashboards, reports, or summaries where columns represent categories (e.g., months, product types).
* **Unpivot** → For feeding data into analytical models, or transforming wide data into long format for easier aggregation/filtering.

---
